## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy

In [2]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [3]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [4]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: c3f6f65661344248bc1da0e1350608d0>,[DO1 data],"Dataset of 6 samples, 1 feature",<class 'torch.Tensor'>


In [5]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 117c98a37d9b4b82844eccbe670eedfe>,[DO2 data],"Dataset of 5 samples, 1 feature",<class 'torch.Tensor'>


## Linear regression

In [6]:
data1_ptr = duet1.store[0]
data2_ptr = duet2.store[0]

print(data1_ptr)
print(data2_ptr)

### Create Base Model

In [7]:
import torch

In [8]:
in_dim = 1
out_dim = 1

In [9]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

### Training

In [10]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [11]:
import torch as th
import numpy as np

In [12]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [13]:
remote_model1 = base_model.send(duet1)

In [14]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

Dummy target data

In [15]:
target1_ptr = th.FloatTensor(np.array([5, 10, 15, 22, 30, 38]).reshape(-1, 1))
target1_ptr

tensor([[ 5.],
        [10.],
        [15.],
        [22.],
        [30.],
        [38.]])

In [16]:
iteration = 10
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 11.002963066101074
Epoch 1 loss 1.7552738189697266
Epoch 2 loss 7.376514434814453
Epoch 3 loss 6.467600345611572
Epoch 4 loss 2.682744026184082
Epoch 5 loss 1.642000675201416
Epoch 6 loss 3.469860792160034
Epoch 7 loss 4.670438289642334
Epoch 8 loss 3.7360055446624756
Epoch 9 loss 2.1136934757232666


Train on Data Owner 2 data

In [17]:
remote_model2 = base_model.send(duet2)

In [18]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

Dummy Target data

In [19]:
target2_ptr = th.FloatTensor(np.array([35, 40, 45, 55, 60]).reshape(-1, 1))
target2_ptr

tensor([[35.],
        [40.],
        [45.],
        [55.],
        [60.]])

In [20]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 12.482412338256836
Epoch 1 loss 24.38145637512207
Epoch 2 loss 10.267278671264648
Epoch 3 loss 3.181546926498413
Epoch 4 loss 12.056376457214355
Epoch 5 loss 11.429590225219727
Epoch 6 loss 4.291439056396484
Epoch 7 loss 3.254056930541992
Epoch 8 loss 7.65777063369751
Epoch 9 loss 8.434379577636719


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [21]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[0.5843]], requires_grad=True), Parameter containing:
tensor([-0.3011], requires_grad=True)]

Remote model1 parameters:
[Parameter containing:
tensor([[0.6736]], requires_grad=True), Parameter containing:
tensor([-0.1736], requires_grad=True)]

Remote model2 parameters:
[Parameter containing:
tensor([[0.6441]], requires_grad=True), Parameter containing:
tensor([-0.2993], requires_grad=True)]


As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [22]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('linear.weight', tensor([[0.6736]])), ('linear.bias', tensor([-0.1736]))])


In [23]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('linear.weight', tensor([[0.6441]])), ('linear.bias', tensor([-0.2993]))])


In [24]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [25]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

OrderedDict([('linear.weight', tensor([[0.6589]])), ('linear.bias', tensor([-0.2365]))])


### Load aggregated weights

In [26]:
combined_model = SyNet(torch)

In [27]:
combined_model.load_state_dict(avg_updates)

In [28]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [29]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [30]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 10.964383125305176 Ground Truth: 12.0
Prediction: 16.23537254333496 Ground Truth: 15.0
Prediction: 20.84748649597168 Ground Truth: 20.0
Prediction: 32.707210540771484 Ground Truth: 30.0
Prediction: 52.47341537475586 Ground Truth: 50.0


In [31]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 9.631661415100098 Ground Truth: 12.0
Prediction: 14.305903434753418 Ground Truth: 15.0
Prediction: 18.39586639404297 Ground Truth: 20.0
Prediction: 28.91291046142578 Ground Truth: 30.0
Prediction: 46.44131851196289 Ground Truth: 50.0


## Comparison to classical linear regression on centralised data

In [32]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [34]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [35]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [36]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [37]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

Epoch 0 loss 2901.619140625
Epoch 10 loss 3.124809980392456
Epoch 20 loss 219.16610717773438
Epoch 30 loss 4.520823955535889
Epoch 40 loss 28.935035705566406
Epoch 50 loss 6.699763774871826
Epoch 60 loss 4.039628028869629
Epoch 70 loss 4.378211498260498
Epoch 80 loss 3.0881948471069336
Epoch 90 loss 3.010854482650757


In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)